In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col, desc, \
                                        rank, dense_rank, sum as f_sum, lead, \
                                            min as f_min, datediff, to_date, row_number \
                                            ,collect_list, date_add       
from pyspark.sql.window import Window

In [2]:
spark = SparkSession.builder.appName('de-interview').getOrCreate()

24/10/04 11:29:03 WARN Utils: Your hostname, ilyoss-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.0.104 instead (on interface en0)
24/10/04 11:29:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/04 11:29:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


24/10/04 11:29:18 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [37]:
badminton_court_data = [
    (1,2,"2016-03-01",5),
    (1,2,"2016-03-02",6),
    (2,3,"2017-06-25",1),
    (3,1,"2016-03-02",2),
    (3,4,"2016-03-02",3),
    (3,2,"2018-07-03",5)
]
coulumns=["user_id","kit_id","login_date","session_count"]
court_input_df = spark.createDataFrame(badminton_court_data,coulumns)
court_input_df.printSchema()
court_input_df.show()

root
 |-- user_id: long (nullable = true)
 |-- kit_id: long (nullable = true)
 |-- login_date: string (nullable = true)
 |-- session_count: long (nullable = true)

+-------+------+----------+-------------+
|user_id|kit_id|login_date|session_count|
+-------+------+----------+-------------+
|      1|     2|2016-03-01|            5|
|      1|     2|2016-03-02|            6|
|      2|     3|2017-06-25|            1|
|      3|     1|2016-03-02|            2|
|      3|     4|2016-03-02|            3|
|      3|     2|2018-07-03|            5|
+-------+------+----------+-------------+



In [141]:
transformed_df=court_input_df.withColumn('login_date', to_date(col("login_date"), "yyyy-MM-dd"))
transformed_df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- kit_id: long (nullable = true)
 |-- login_date: date (nullable = true)
 |-- session_count: long (nullable = true)



In [142]:
court_input_df.createOrReplaceTempView("court_input")
result_df=spark.sql("""select user_id, 
                    kit_id, cast(login_date as DATE) as login_date, 
                    session_count from court_input""")
result_df.printSchema()

root
 |-- user_id: long (nullable = true)
 |-- kit_id: long (nullable = true)
 |-- login_date: date (nullable = true)
 |-- session_count: long (nullable = true)



In [143]:
result_df = spark.sql("""
    select user_id, min(login_date) from court_input group by user_id      
""")
result_df.show()

+-------+---------------+
|user_id|min(login_date)|
+-------+---------------+
|      1|     2016-03-01|
|      2|     2017-06-25|
|      3|     2016-03-02|
+-------+---------------+



In [43]:
new_court_input_df = court_input_df.groupBy("user_id").agg({"login_date":"min"})
new_court_input_df.show()

+-------+---------------+
|user_id|min(login_date)|
+-------+---------------+
|      1|     2016-03-01|
|      2|     2017-06-25|
|      3|     2016-03-02|
+-------+---------------+



In [144]:
result_df = spark.sql("""
     with cte as (
                      select *, rank(login_date) over (partition by user_id order by login_date) as first_date 
                      from court_input)                 
    select user_id, login_date as first_login_date from cte where first_date=1 group by user_id, login_date
""")
result_df.show()

+-------+----------------+
|user_id|first_login_date|
+-------+----------------+
|      1|      2016-03-01|
|      2|      2017-06-25|
|      3|      2016-03-02|
+-------+----------------+



In [52]:
result_df = spark.sql("""
     with cte as (
                      select *, rank(login_date) over (partition by user_id order by login_date) as first_date 
                      from court_input)                 
    select user_id, kit_id, login_date as first_login_date from cte where first_date=1
""")
result_df.show()

+-------+------+----------------+
|user_id|kit_id|first_login_date|
+-------+------+----------------+
|      1|     2|      2016-03-01|
|      2|     3|      2017-06-25|
|      3|     1|      2016-03-02|
|      3|     4|      2016-03-02|
+-------+------+----------------+



In [145]:
transformed_df.show()
transformed_df.printSchema()

+-------+------+----------+-------------+
|user_id|kit_id|login_date|session_count|
+-------+------+----------+-------------+
|      1|     2|2016-03-01|            5|
|      1|     2|2016-03-02|            6|
|      2|     3|2017-06-25|            1|
|      3|     1|2016-03-02|            2|
|      3|     4|2016-03-02|            3|
|      3|     2|2018-07-03|            5|
+-------+------+----------+-------------+

root
 |-- user_id: long (nullable = true)
 |-- kit_id: long (nullable = true)
 |-- login_date: date (nullable = true)
 |-- session_count: long (nullable = true)



In [146]:
windowSpec = Window.partitionBy("user_id").orderBy("login_date")
operations_df=transformed_df.withColumn("row_number",row_number().over(windowSpec)).filter(col("row_number")==1).show()

+-------+------+----------+-------------+----------+
|user_id|kit_id|login_date|session_count|row_number|
+-------+------+----------+-------------+----------+
|      1|     2|2016-03-01|            5|         1|
|      2|     3|2017-06-25|            1|         1|
|      3|     1|2016-03-02|            2|         1|
+-------+------+----------+-------------+----------+



In [147]:
operations_df2=transformed_df.withColumn("dense_rank",dense_rank() \
                                         .over(windowSpec)) \
                                            .filter(col("dense_rank")==1) \
                                                .select("user_id","kit_id")
operations_df2.show()


+-------+------+
|user_id|kit_id|
+-------+------+
|      1|     2|
|      2|     3|
|      3|     1|
|      3|     4|
+-------+------+



In [148]:
operations_df3 = operations_df2.groupBy("user_id").agg(collect_list("kit_id").alias("kit_ids"))
operations_df3.show()

+-------+-------+
|user_id|kit_ids|
+-------+-------+
|      1|    [2]|
|      2|    [3]|
|      3| [1, 4]|
+-------+-------+



In [149]:
transformed_df.printSchema()
transformed_df.show()

root
 |-- user_id: long (nullable = true)
 |-- kit_id: long (nullable = true)
 |-- login_date: date (nullable = true)
 |-- session_count: long (nullable = true)

+-------+------+----------+-------------+
|user_id|kit_id|login_date|session_count|
+-------+------+----------+-------------+
|      1|     2|2016-03-01|            5|
|      1|     2|2016-03-02|            6|
|      2|     3|2017-06-25|            1|
|      3|     1|2016-03-02|            2|
|      3|     4|2016-03-02|            3|
|      3|     2|2018-07-03|            5|
+-------+------+----------+-------------+



In [150]:
windowSpec = Window.partitionBy("user_id").orderBy("login_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
operations_df4=transformed_df.withColumn("games_played",f_sum("session_count").over(windowSpec))
operations_df4.show()

+-------+------+----------+-------------+------------+
|user_id|kit_id|login_date|session_count|games_played|
+-------+------+----------+-------------+------------+
|      1|     2|2016-03-01|            5|           5|
|      1|     2|2016-03-02|            6|          11|
|      2|     3|2017-06-25|            1|           1|
|      3|     1|2016-03-02|            2|           2|
|      3|     4|2016-03-02|            3|           5|
|      3|     2|2018-07-03|            5|          10|
+-------+------+----------+-------------+------------+



In [151]:
result_df = spark.sql("""

                      select *, sum(session_count) over (partition by user_id order by login_date
                      ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW) as games_played from court_input
""")
result_df.show()

+-------+------+----------+-------------+------------+
|user_id|kit_id|login_date|session_count|games_played|
+-------+------+----------+-------------+------------+
|      1|     2|2016-03-01|            5|           5|
|      1|     2|2016-03-02|            6|          11|
|      2|     3|2017-06-25|            1|           1|
|      3|     1|2016-03-02|            2|           2|
|      3|     4|2016-03-02|            3|           5|
|      3|     2|2018-07-03|            5|          10|
+-------+------+----------+-------------+------------+



In [152]:
result_df=spark.sql("""
                    with cte as (
                        select *, dateadd(day,1,login_date) next_day from court_input
                    )select c.user_id, c.kit_id,c.login_date,c.session_count
                    from cte
                    join court_input c
                    on c.user_id=cte.user_id
                    and c.login_date=cte.next_day
                        
""")
result_df.show()

+-------+------+----------+-------------+
|user_id|kit_id|login_date|session_count|
+-------+------+----------+-------------+
|      1|     2|2016-03-02|            6|
+-------+------+----------+-------------+



In [154]:
windowSpec = Window.partitionBy("user_id").orderBy("login_date").rowsBetween(Window.unboundedPreceding, Window.currentRow)
operations_df5 = transformed_df.withColumn("next_day", date_add(transformed_df["login_date"], 1))
operations_df6 = (
    operations_df5.alias("tbl1")  # First instance of transformed_df
    .join(transformed_df.alias("tbl2"), 
          (operations_df5["next_day"] == col("tbl2.login_date")) 
          &(operations_df5["user_id"] == col("tbl2.user_id"))
          , "inner")  # Use qualified names
)
operations_df6.show()


+-------+------+----------+-------------+----------+-------+------+----------+-------------+
|user_id|kit_id|login_date|session_count|  next_day|user_id|kit_id|login_date|session_count|
+-------+------+----------+-------------+----------+-------+------+----------+-------------+
|      1|     2|2016-03-01|            5|2016-03-02|      1|     2|2016-03-02|            6|
+-------+------+----------+-------------+----------+-------+------+----------+-------------+



24/10/03 20:37:51 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 262421 ms exceeds timeout 120000 ms
24/10/03 20:37:51 WARN SparkContext: Killing executors is not supported by current scheduler.
24/10/03 20:37:57 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$